In [846]:
import pandas as pd
import re
from functools import reduce
import sys
import csv
import glob
import os
from gensim.summarization.summarizer import summarize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [157]:
path=r"D:\LINKEDIN_DATA\data\excel"
files = os.listdir(path)
files

['karu_profiles.xlsx',
 'seethu_profiles.xlsx',
 'seethu_profiles_1.xlsx',
 'sowmi_profiles.xlsx']

In [158]:
files_xlsx = [f for f in files if f[-4:] == 'xlsx']
files_xlsx

['karu_profiles.xlsx',
 'seethu_profiles.xlsx',
 'seethu_profiles_1.xlsx',
 'sowmi_profiles.xlsx']

In [159]:
df_rec = pd.DataFrame()

In [160]:
for f in files_xlsx:
    data = pd.read_excel(path+'/'+f)
    df_rec = df_rec.append(data,ignore_index=True)
df_rec.shape

(1095, 9)

In [161]:
df_1=pd.concat(g for _, g in df_rec.groupby("URL") if len(g) ==1)
df_1.shape

(1095, 9)

In [162]:
df_1=pd.concat(g for _, g in df_rec.groupby("Experience_1") if len(g) ==1)
df_1.shape

(1041, 9)

In [163]:
df_1.isna().sum()

Unnamed: 0      0
Name            0
Summary         0
Experience_1    0
Experience_2    0
Education_1     0
Education_2     0
Skills          0
URL             0
dtype: int64

In [164]:
df_1['Domain']=''

In [165]:
df_2=pd.read_excel('D:\LINKEDIN_DATA\APRIL_13_2020_EXCELDATA\correctdata.xlsx')

In [166]:
df=df_2.append(df_1)

In [167]:
df.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
             986,  624,  280,  402,  104,  580,  809, 1004,  954, 1002],
           dtype='int64', length=5869)

In [168]:
df.shape

(5869, 11)

In [169]:
df.drop(['Unnamed: 0'],inplace=True,axis=1)

In [170]:
df.drop(['Unnamed: 0.1'],inplace=True,axis=1)

In [171]:
df.isna().sum()

Domain          2644
Education_1     6   
Education_2     33  
Experience_1    0   
Experience_2    0   
Name            0   
Skills          0   
Summary         0   
URL             0   
dtype: int64

In [172]:
def expconv2months(x):
    try:
        a=x.split('\n')
        if 'Total Duration' in a:
            t1=a.index('Total Duration')
        else:
            t1=a.index('Employment Duration')
        dur1=a[t1+1]
        z1=dur1.split(' ')
        n1,n2=0,0
        n1=[int(z1[z1.index(s)-1])*12 for s in z1 if any(xs in s for xs in ['yrs','yr'])]
        n2=[int(z1[z1.index(s)-1]) for s in z1 if any(xs in s for xs in ['mos','mo'])]
        return sum(n1+n2)
    except:
        return 0

In [173]:
def designation(x):
    try:
        a=x.split('\n')  
    except:
        res='Not Specified'
        return res
    else:
        if a:
            if 'Title' in a:
                t=a.index('Title')
                res=a[t+1]
            else:
                res=a[0]
        return res

In [174]:
def educonv2months(x):
    try:
        N=re.findall(r'\d{4} – \d{4}',x)
        M=N[0].split('–')
        a=int(M[0])
        b=int(M[1])  
        res=b-a
        return res
    except:
        return 0

In [175]:
def Total(x,y):
    sum=x+y
    return sum

In [176]:
def current_organization(x):
    try:
        a=x.split('\n')  
    except:
        res='Not Specified'
        return res
    else:
        if a:
            if 'Company Name' in a:
                t=a.index('Company Name')
                res=a[t+1]
            else:
                res='Not Specified'
                return res
        return res

In [177]:
df['Designation_1']=df.apply(lambda row:designation(row['Experience_1']) if row['Experience_1']!='Company Name' else\
                             lambda row:designation(row['Experience_2']),axis=1)
df['Designation_2']=df.apply(lambda row:designation(row['Experience_2']),axis=1)
df['Current_Organization']=df.apply(lambda row:current_organization(row['Experience_1']) if row['Experience_1']!='Company Name' else\
                                    lambda row:current_organization(row['Experience_2']),axis=1)
df['ExpDurationInMonths_1']=df.apply(lambda row:expconv2months(row['Experience_1']),axis=1)
df['ExpDurationInMonths_2']=df.apply(lambda row:expconv2months(row['Experience_2']),axis=1)
df['EduDurationInMonths_1']=df.apply(lambda row:educonv2months(row['Education_1']),axis=1)
df['EduDurationInMonths_2']=df.apply(lambda row:educonv2months(row['Education_2']),axis=1)

In [178]:
final_df=pd.DataFrame(columns=['Name','Summary','Skills','Experience_1','Current_Organization','Designation_1','ExpDurationInMonths_1','Experience_2',\
                            'Designation_2','ExpDurationInMonths_2','Education_1','EduDurationInMonths_1', 'Education_2',\
                            'EduDurationInMonths_2','Total_Exp','Total_Edu', 'URL','Domain'])
DATA=final_df.append(df)
DATA=DATA.reindex(final_df.columns, axis=1)

In [179]:
DATA['ExpDurationInMonths_1']=DATA['ExpDurationInMonths_1'].astype("Float32").astype("Int32")
DATA['ExpDurationInMonths_2']=DATA['ExpDurationInMonths_2'].astype("Float32").astype("Int32")
DATA['EduDurationInMonths_1']=DATA['EduDurationInMonths_1'].astype("Float32").astype("Int32")
DATA['EduDurationInMonths_2']=DATA['EduDurationInMonths_2'].astype("Float32").astype("Int32")
DATA['Total_Exp']=DATA.apply(lambda row:Total(row['ExpDurationInMonths_1'],row['ExpDurationInMonths_2']),axis=1)
DATA['Total_Edu']=DATA.apply(lambda row:Total(row['EduDurationInMonths_1'],row['EduDurationInMonths_2']),axis=1)
DATA['Total_Exp']=DATA['Total_Exp'].astype("Float32").astype("Int32")
DATA['Total_Edu']=DATA['Total_Edu'].astype("Float32").astype("Int32")

In [180]:
#DATA[DATA['Name']=='Aniruddha Khekale']

In [181]:
#pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
DATA['New Summary']=''

In [ ]:
#Function to clean text from Summary section
def clean_text(text):
    for ch in ['\n','!','•','*','\uf072','\uf03d','\uf0d8','\uf076','\uf095\uf020','\uf076','\uf0d8','➢','●','{','}','\uf0d8','_______________________________________ ','\uf0dc','•','♦','✪','\uf0a4','●','►','\uf02a','•','⇛','\uf0a7','●','~The Entrepreneur👨\u200d💼','\uf0b2','✪','__________________________________________________________________','■','►']:
        if ch in text:
            text = text.replace(ch," ")
    return text

In [ ]:
DATA['New Summary']=DATA.apply(lambda row:clean_text(row['Summary']),axis=1)

In [ ]:
#Function to remove unicode characters from Summary section
def remove_unicode(text):
    New_text=re.sub(r'[^\x00-\x7f]',r' ',text)
    return New_text

In [ ]:
DATA['Latest_Summary']=''

In [ ]:
DATA['Latest_Summary']=DATA.apply(lambda row:remove_unicode(row['New Summary']),axis=1)

In [677]:
#Remove some more characters
def remove_chars(text):
    for ch in ['-->','#','-','------------------------------------------------------','~','- >','____','//','>','=================================','________________________']:
        if ch in text:
            text = text.replace(ch," ")
    return text

In [676]:
DATA['Summary_Final']=''

In [678]:
DATA['Summary_Final']=DATA.apply(lambda row:remove_chars(row['Latest_Summary']),axis=1)

In [682]:
#Replace 'see more with 'click on more to see more'
def replace_text(text):
    New_text=text.replace('. see more',"Click on more to see more")
    return New_text

In [683]:
DATA['Summary_Final']=DATA.apply(lambda row:replace_text(row['Summary_Final']),axis=1)

In [716]:
#Remove Multiple white spaces
DATA['Summary_Final']=(DATA['Summary_Final'].str.split()).str.join(' ')

In [690]:
#Capitalize the first letter of Summary section in all rows
DATA['Summary_Final']= DATA['Summary_Final'].str.capitalize()

In [ ]:
#Ensure text in Summary ends with a fullstop
#DATA['Summary'].iloc[7][:-1]+'.'

In [705]:
#Replace 'i.e' with space
def replace_ie(text):
    New_text=text.replace('i.e.'," ")
    return New_text

In [706]:
DATA['Summary_Final']=DATA.apply(lambda row:replace_ie(row['Summary_Final']),axis=1)

In [722]:
#Capitalize first letter of every senetnce
def cap(text):
    New_text='.'.join(map(lambda s: s.strip().capitalize(), text.split('.')))
    return New_text

In [723]:
DATA['Summary_Final']=DATA.apply(lambda row:cap(row['Summary_Final']),axis=1)

In [712]:
#Replace Multiple full stops with single full stop
def mulstops(text):
    text_replaced = re.sub(r'\.+', ".", text)
    return text_replaced

In [713]:
DATA['Summary_Final']=DATA.apply(lambda row:mulstops(row['Summary_Final']),axis=1)

In [772]:
#DATA.drop(['New Summary','Latest_Summary'],axis=1,inplace=True)

In [773]:
DATA.columns

Index(['Name', 'Summary', 'Skills', 'Experience_1', 'Current_Organization',
       'Designation_1', 'ExpDurationInMonths_1', 'Experience_2',
       'Designation_2', 'ExpDurationInMonths_2', 'Education_1',
       'EduDurationInMonths_1', 'Education_2', 'EduDurationInMonths_2',
       'Total_Exp', 'Total_Edu', 'URL', 'Domain', 'Summary_Final'],
      dtype='object')

In [779]:
DATA['Summarized_Text']=''

In [861]:
#Function for text summarization up to 150 words if number of sentences in Summary is > 10
def summ(text):
    from nltk.tokenize import sent_tokenize
    from gensim.summarization import summarize
    sen=text
    #res = len(re.findall(r'\w+',sen)) 
    number_of_sentences = len(sent_tokenize(sen))
    if number_of_sentences > 10:
        new_text=summarize(sen,word_count=150)
        return new_text
    else:
        return sen

In [862]:
from gensim.summarization import summarize
DATA['Summarized_Text']=DATA.apply(lambda row:summ(row['Summary']),axis=1)

In [872]:
DATA['Summarized_Text']=DATA.apply(lambda row:clean_text(row['Summarized_Text']),axis=1)
DATA['Summarized_Text']=DATA.apply(lambda row:remove_unicode(row['Summarized_Text']),axis=1)
DATA['Summarized_Text']=DATA.apply(lambda row:remove_chars(row['Summarized_Text']),axis=1)
DATA['Summarized_Text']=DATA.apply(lambda row:replace_text(row['Summarized_Text']),axis=1)
DATA['Summarized_Text']=(DATA['Summarized_Text'].str.split()).str.join(' ')
DATA['Summarized_Text']= DATA['Summarized_Text'].str.capitalize()
DATA['Summarized_Text']=DATA.apply(lambda row:replace_ie(row['Summarized_Text']),axis=1)
DATA['Summarized_Text']=DATA.apply(lambda row:cap(row['Summarized_Text']),axis=1)
DATA['Summarized_Text']=DATA.apply(lambda row:mulstops(row['Summarized_Text']),axis=1)

In [874]:
DATA['Summarized_Text'].iloc[11]

'Having had many years in purchasing, warehousing, logistics, staff management and operational leadership roles with continued success in meeting business/operational goals, i believe i can make a valuable contribution to your organisation s future projects and initiatives.What i bring to this position is a combination of team leadership, strategic planning, and business development skills that i have leveraged to meet and exceed expectations.I have analysed core operational/business functions and implemented the most appropriate solutions to drive continual business growth.I have demonstrated proficiency in all core office and warehouse administration and operational functions, including document preparation, finance, hr, oh & s, and internal/external stakeholder communications.I have experience on the use all microsoft office products, and implementation and key user operation and management of sap b1/sap erp, wintergrate/eclipse, stamina and epicor v7/v9.In previous positions, i hav

In [877]:
DATA.shape

(5857, 20)

In [878]:
DATA.isna().sum()

Name                     0   
Summary                  0   
Skills                   0   
Experience_1             0   
Current_Organization     0   
Designation_1            0   
ExpDurationInMonths_1    0   
Experience_2             0   
Designation_2            0   
ExpDurationInMonths_2    0   
Education_1              6   
EduDurationInMonths_1    0   
Education_2              33  
EduDurationInMonths_2    0   
Total_Exp                0   
Total_Edu                0   
URL                      0   
Domain                   2639
Summary_Final            0   
Summarized_Text          0   
dtype: int64

In [879]:
DATA.to_excel(r'E:\CAPSTONE\FILES TO BE UPLOADED\DatsetUsedForTraining.xlsx')

In [751]:
#DATA['Summary_Final'].to_excel(r'C:\Users\user\Desktop\Written\text5.xlsx')

In [679]:
#DATA[['Summary','Summary_Final']].to_excel(r'C:\Users\user\Desktop\Written\text3.xlsx')

In [185]:
#pd.set_option('display.max_colwidth', -1)

In [ ]:
#DATA['Latest_Summary'].iloc[3071]

In [ ]:
#DATA[DATA['Summary']==0]
#DATA.drop(DATA.index[[391,756]],inplace=True)

In [515]:
#DATA[['Summary','New Summary']].to_excel(r'C:\Users\user\Desktop\Written\text.xlsx')

In [537]:
#DATA['New Summary'].iloc[3308]=DATA['New Summary'].iloc[2816].replace(' o ',' ')